# Setup & Import Libraries

In [1]:
! pip install -q kaggle

In [2]:
from google.colab import files
import os
import numpy as np
import pandas as pd
import datetime

import tensorflow as tf

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# Download the Dataset

In [ ]:
files.upload() #upload kaggle.json.

In [4]:
! mkdir ~/.kaggle 
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d theforcecoder/wind-power-forecasting
!unzip wind-power-forecasting.zip

os.remove('wind-power-forecasting.zip')  

 93% 7.00M/7.51M [00:00<00:00, 72.3MB/s]
100% 7.51M/7.51M [00:00<00:00, 75.5MB/s]
Archive:  wind-power-forecasting.zip
  inflating: Turbine_Data.csv        


# Load Dataset

In [5]:
df = pd.read_csv('Turbine_Data.csv', parse_dates = True, header = 0)
df['Time'] = pd.to_datetime(df["Unnamed: 0"], yearfirst = True)

In [6]:
df.head()

,Unnamed: 0,ActivePower,AmbientTemperatue,BearingShaftTemperature,Blade1PitchAngle,Blade2PitchAngle,Blade3PitchAngle,ControlBoxTemperature,GearboxBearingTemperature,GearboxOilTemperature,...,HubTemperature,MainBoxTemperature,NacellePosition,ReactivePower,RotorRPM,TurbineStatus,WTG,WindDirection,WindSpeed,Time
0,2017-12-31 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,G01,NaN,NaN,2017-12-31 00:00:00+00:00
1,2017-12-31 00:10:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,G01,NaN,NaN,2017-12-31 00:10:00+00:00
2,2017-12-31 00:20:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,G01,NaN,NaN,2017-12-31 00:20:00+00:00
3,2017-12-31 00:30:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,G01,NaN,NaN,2017-12-31 00:30:00+00:00
4,2017-12-31 00:40:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,G01,NaN,NaN,2017-12-31 00:40:00+00:00


# Exploratory Data Analysis

In [7]:
df.describe()

,ActivePower,AmbientTemperatue,BearingShaftTemperature,Blade1PitchAngle,Blade2PitchAngle,Blade3PitchAngle,ControlBoxTemperature,GearboxBearingTemperature,GearboxOilTemperature,GeneratorRPM,GeneratorWinding1Temperature,GeneratorWinding2Temperature,HubTemperature,MainBoxTemperature,NacellePosition,ReactivePower,RotorRPM,TurbineStatus,WindDirection,WindSpeed
count,94750.000000,93817.000000,62518.000000,41996.000000,41891.000000,41891.000000,62160.0,62540.000000,62438.000000,62295.000000,62427.000000,62449.000000,62406.000000,62507.000000,72278.000000,94748.000000,62127.000000,6.290800e+04,72278.000000,94595.000000
mean,619.109805,28.774654,43.010189,9.749641,10.036535,10.036535,0.0,64.234170,57.561217,1102.026269,72.460403,71.826659,36.897978,39.547603,196.290539,88.133966,9.907500,2.280429e+03,196.290539,5.878960
std,611.275373,4.369145,5.545312,20.644828,20.270465,20.270465,0.0,10.455556,6.323895,528.063946,22.627489,22.650255,5.178711,5.732783,88.296554,116.596725,4.718421,3.586034e+05,88.296554,2.619084
min,-38.524659,0.000000,0.000000,-43.156734,-26.443415,-26.443415,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-203.182591,0.000000,0.000000e+00,0.000000,0.000000
25%,79.642258,25.627428,39.840247,-0.939849,-0.433264,-0.433264,0.0,57.872242,53.942181,1029.812177,55.492241,54.763998,33.943949,35.812500,145.000000,-0.432137,9.231091,2.000000e+00,145.000000,3.823330
50%,402.654893,28.340541,42.910877,0.394399,0.888977,0.888977,0.0,64.834662,57.196089,1124.860720,65.788800,65.004946,37.003815,39.491310,182.000000,35.883659,10.098702,2.000000e+00,182.000000,5.557765
75%,1074.591780,31.664772,47.007976,8.099302,8.480194,8.480194,0.0,71.079306,61.305312,1515.402005,85.867449,85.337740,40.008425,43.359375,271.000000,147.359075,13.600413,2.000000e+00,271.000000,7.506710
max,1779.032433,42.405597,55.088655,90.143610,90.017830,90.017830,0.0,82.237932,70.764581,1809.941700,126.773031,126.043018,47.996185,54.250000,357.000000,403.713620,16.273495,6.574653e+07,357.000000,22.970893


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118224 entries, 0 to 118223
Data columns (total 23 columns):
 #   Column                        Non-Null Count   Dtype              
---  ------                        --------------   -----              
 0   Unnamed: 0                    118224 non-null  object             
 1   ActivePower                   94750 non-null   float64            
 2   AmbientTemperatue             93817 non-null   float64            
 3   BearingShaftTemperature       62518 non-null   float64            
 4   Blade1PitchAngle              41996 non-null   float64            
 5   Blade2PitchAngle              41891 non-null   float64            
 6   Blade3PitchAngle              41891 non-null   float64            
 7   ControlBoxTemperature         62160 non-null   float64            
 8   GearboxBearingTemperature     62540 non-null   float64            
 9   GearboxOilTemperature         62438 non-null   float64            
 10  GeneratorRPM        

**Handle with missing values**

In [9]:
df.isnull().sum()

Unnamed: 0                          0
ActivePower                     23474
AmbientTemperatue               24407
BearingShaftTemperature         55706
Blade1PitchAngle                76228
Blade2PitchAngle                76333
Blade3PitchAngle                76333
ControlBoxTemperature           56064
GearboxBearingTemperature       55684
GearboxOilTemperature           55786
GeneratorRPM                    55929
GeneratorWinding1Temperature    55797
GeneratorWinding2Temperature    55775
HubTemperature                  55818
MainBoxTemperature              55717
NacellePosition                 45946
ReactivePower                   23476
RotorRPM                        56097
TurbineStatus                   55316
WTG                                 0
WindDirection                   45946
WindSpeed                       23629
Time                                0
dtype: int64

In [10]:
df = df.fillna(method="ffill").fillna(method="bfill")

In [11]:
df.isnull().sum()

Unnamed: 0                      0
ActivePower                     0
AmbientTemperatue               0
BearingShaftTemperature         0
Blade1PitchAngle                0
Blade2PitchAngle                0
Blade3PitchAngle                0
ControlBoxTemperature           0
GearboxBearingTemperature       0
GearboxOilTemperature           0
GeneratorRPM                    0
GeneratorWinding1Temperature    0
GeneratorWinding2Temperature    0
HubTemperature                  0
MainBoxTemperature              0
NacellePosition                 0
ReactivePower                   0
RotorRPM                        0
TurbineStatus                   0
WTG                             0
WindDirection                   0
WindSpeed                       0
Time                            0
dtype: int64

In [12]:
df.describe()

,ActivePower,AmbientTemperatue,BearingShaftTemperature,Blade1PitchAngle,Blade2PitchAngle,Blade3PitchAngle,ControlBoxTemperature,GearboxBearingTemperature,GearboxOilTemperature,GeneratorRPM,GeneratorWinding1Temperature,GeneratorWinding2Temperature,HubTemperature,MainBoxTemperature,NacellePosition,ReactivePower,RotorRPM,TurbineStatus,WindDirection,WindSpeed
count,118224.000000,118224.000000,118224.000000,118224.000000,118224.000000,118224.000000,118224.0,118224.000000,118224.000000,118224.000000,118224.000000,118224.000000,118224.000000,118224.000000,118224.000000,118224.000000,118224.000000,1.182240e+05,118224.000000,118224.000000
mean,639.934548,28.574250,44.966830,31.797893,30.718228,30.718228,0.0,68.885245,60.146597,1359.868503,87.239668,86.582197,38.700162,40.595611,201.719506,84.387076,12.199976,1.239299e+03,201.719506,6.031632
std,630.144323,4.308120,6.404031,22.150405,21.173546,21.173546,0.0,11.675803,5.812945,527.578035,25.175172,25.170463,5.820204,5.925493,90.954878,116.505296,4.737852,2.615880e+05,90.954878,2.819626
min,-38.524659,0.000000,0.000000,-43.156734,-26.443415,-26.443415,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-203.182591,0.000000,0.000000e+00,0.000000,0.000000
25%,70.595235,25.531339,41.608761,1.737424,2.209115,2.209115,0.0,62.865522,56.353738,1049.551193,62.588232,61.782429,35.994478,38.270834,145.000000,-0.502655,9.410517,0.000000e+00,145.000000,3.866613
50%,423.445880,28.126315,47.901936,45.736893,43.699357,43.699357,0.0,75.066281,63.304417,1691.274462,98.608170,97.978180,39.996185,41.898426,188.000000,28.384563,15.160864,2.000000e+00,188.000000,5.672165
75%,1155.626617,31.261132,47.901936,45.736893,43.699357,43.699357,0.0,77.119133,64.204399,1751.715563,113.290871,112.592163,42.996094,44.000000,280.000000,137.433198,15.708135,2.000000e+00,280.000000,7.791896
max,1779.032433,42.405597,55.088655,90.143610,90.017830,90.017830,0.0,82.237932,70.764581,1809.941700,126.773031,126.043018,47.996185,54.250000,357.000000,403.713620,16.273495,6.574653e+07,357.000000,22.970893


**We can see in describe method that features: `Blade2PitchAngle` and `Blade3PitchAngle` having the same values for descritive stat. So we can drop one column**

In [13]:
def handle_equals_values(feature_1, feature_2):
    global df
    if (df[feature_1].equals(df[feature_2]) == True):
        df = df.drop(feature_2, axis=1)

In [14]:
handle_equals_values("Blade2PitchAngle", "Blade3PitchAngle")

In [15]:
df.describe()

,ActivePower,AmbientTemperatue,BearingShaftTemperature,Blade1PitchAngle,Blade2PitchAngle,ControlBoxTemperature,GearboxBearingTemperature,GearboxOilTemperature,GeneratorRPM,GeneratorWinding1Temperature,GeneratorWinding2Temperature,HubTemperature,MainBoxTemperature,NacellePosition,ReactivePower,RotorRPM,TurbineStatus,WindDirection,WindSpeed
count,118224.000000,118224.000000,118224.000000,118224.000000,118224.000000,118224.0,118224.000000,118224.000000,118224.000000,118224.000000,118224.000000,118224.000000,118224.000000,118224.000000,118224.000000,118224.000000,1.182240e+05,118224.000000,118224.000000
mean,639.934548,28.574250,44.966830,31.797893,30.718228,0.0,68.885245,60.146597,1359.868503,87.239668,86.582197,38.700162,40.595611,201.719506,84.387076,12.199976,1.239299e+03,201.719506,6.031632
std,630.144323,4.308120,6.404031,22.150405,21.173546,0.0,11.675803,5.812945,527.578035,25.175172,25.170463,5.820204,5.925493,90.954878,116.505296,4.737852,2.615880e+05,90.954878,2.819626
min,-38.524659,0.000000,0.000000,-43.156734,-26.443415,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-203.182591,0.000000,0.000000e+00,0.000000,0.000000
25%,70.595235,25.531339,41.608761,1.737424,2.209115,0.0,62.865522,56.353738,1049.551193,62.588232,61.782429,35.994478,38.270834,145.000000,-0.502655,9.410517,0.000000e+00,145.000000,3.866613
50%,423.445880,28.126315,47.901936,45.736893,43.699357,0.0,75.066281,63.304417,1691.274462,98.608170,97.978180,39.996185,41.898426,188.000000,28.384563,15.160864,2.000000e+00,188.000000,5.672165
75%,1155.626617,31.261132,47.901936,45.736893,43.699357,0.0,77.119133,64.204399,1751.715563,113.290871,112.592163,42.996094,44.000000,280.000000,137.433198,15.708135,2.000000e+00,280.000000,7.791896
max,1779.032433,42.405597,55.088655,90.143610,90.017830,0.0,82.237932,70.764581,1809.941700,126.773031,126.043018,47.996185,54.250000,357.000000,403.713620,16.273495,6.574653e+07,357.000000,22.970893


# Preprocessing

**Add date for `Unnamed: 0` column.**



In [16]:
df["Unnamed: 0"] = pd.to_datetime(df["Unnamed: 0"])
df.rename(columns={"Unnamed: 0": "date"}, inplace=True)

In [17]:
df.head()

,date,ActivePower,AmbientTemperatue,BearingShaftTemperature,Blade1PitchAngle,Blade2PitchAngle,ControlBoxTemperature,GearboxBearingTemperature,GearboxOilTemperature,GeneratorRPM,...,HubTemperature,MainBoxTemperature,NacellePosition,ReactivePower,RotorRPM,TurbineStatus,WTG,WindDirection,WindSpeed,Time
0,2017-12-31 00:00:00+00:00,-5.357727,23.148729,47.901936,45.736893,43.699357,0.0,77.119133,64.204399,1751.715563,...,42.996094,41.898426,8.0,-9.96083,15.708135,0.0,G01,8.0,2.279088,2017-12-31 00:00:00+00:00
1,2017-12-31 00:10:00+00:00,-5.357727,23.148729,47.901936,45.736893,43.699357,0.0,77.119133,64.204399,1751.715563,...,42.996094,41.898426,8.0,-9.96083,15.708135,0.0,G01,8.0,2.279088,2017-12-31 00:10:00+00:00
2,2017-12-31 00:20:00+00:00,-5.357727,23.148729,47.901936,45.736893,43.699357,0.0,77.119133,64.204399,1751.715563,...,42.996094,41.898426,8.0,-9.96083,15.708135,0.0,G01,8.0,2.279088,2017-12-31 00:20:00+00:00
3,2017-12-31 00:30:00+00:00,-5.357727,23.148729,47.901936,45.736893,43.699357,0.0,77.119133,64.204399,1751.715563,...,42.996094,41.898426,8.0,-9.96083,15.708135,0.0,G01,8.0,2.279088,2017-12-31 00:30:00+00:00
4,2017-12-31 00:40:00+00:00,-5.357727,23.148729,47.901936,45.736893,43.699357,0.0,77.119133,64.204399,1751.715563,...,42.996094,41.898426,8.0,-9.96083,15.708135,0.0,G01,8.0,2.279088,2017-12-31 00:40:00+00:00


In [18]:
df = df.join(pd.get_dummies(df["date"].dt.dayofweek))
df.head()

,date,ActivePower,AmbientTemperatue,BearingShaftTemperature,Blade1PitchAngle,Blade2PitchAngle,ControlBoxTemperature,GearboxBearingTemperature,GearboxOilTemperature,GeneratorRPM,...,WindDirection,WindSpeed,Time,0,1,2,3,4,5,6
0,2017-12-31 00:00:00+00:00,-5.357727,23.148729,47.901936,45.736893,43.699357,0.0,77.119133,64.204399,1751.715563,...,8.0,2.279088,2017-12-31 00:00:00+00:00,0,0,0,0,0,0,1
1,2017-12-31 00:10:00+00:00,-5.357727,23.148729,47.901936,45.736893,43.699357,0.0,77.119133,64.204399,1751.715563,...,8.0,2.279088,2017-12-31 00:10:00+00:00,0,0,0,0,0,0,1
2,2017-12-31 00:20:00+00:00,-5.357727,23.148729,47.901936,45.736893,43.699357,0.0,77.119133,64.204399,1751.715563,...,8.0,2.279088,2017-12-31 00:20:00+00:00,0,0,0,0,0,0,1
3,2017-12-31 00:30:00+00:00,-5.357727,23.148729,47.901936,45.736893,43.699357,0.0,77.119133,64.204399,1751.715563,...,8.0,2.279088,2017-12-31 00:30:00+00:00,0,0,0,0,0,0,1
4,2017-12-31 00:40:00+00:00,-5.357727,23.148729,47.901936,45.736893,43.699357,0.0,77.119133,64.204399,1751.715563,...,8.0,2.279088,2017-12-31 00:40:00+00:00,0,0,0,0,0,0,1


**Transform the `date` column into periodicity using sin and cos for days and years.**

In [19]:
day = 24*60*60
year = 365.2425*day

timestamp_s = df["date"].map(datetime.datetime.timestamp)

df["sin_day"] = np.sin(timestamp_s * (2 * np.pi / day))
df["cos_day"] = np.cos(timestamp_s * (2 * np.pi / day))
df["sin_year"] = np.sin(timestamp_s * (2 * np.pi / year))
df["cos_year"] = np.cos(timestamp_s * (2 * np.pi / year))

In [20]:
df.head()

,date,ActivePower,AmbientTemperatue,BearingShaftTemperature,Blade1PitchAngle,Blade2PitchAngle,ControlBoxTemperature,GearboxBearingTemperature,GearboxOilTemperature,GeneratorRPM,...,1,2,3,4,5,6,sin_day,cos_day,sin_year,cos_year
0,2017-12-31 00:00:00+00:00,-5.357727,23.148729,47.901936,45.736893,43.699357,0.0,77.119133,64.204399,1751.715563,...,0,0,0,0,0,1,-6.688386e-12,1.000000,-0.011010,0.999939
1,2017-12-31 00:10:00+00:00,-5.357727,23.148729,47.901936,45.736893,43.699357,0.0,77.119133,64.204399,1751.715563,...,0,0,0,0,0,1,4.361939e-02,0.999048,-0.010890,0.999941
2,2017-12-31 00:20:00+00:00,-5.357727,23.148729,47.901936,45.736893,43.699357,0.0,77.119133,64.204399,1751.715563,...,0,0,0,0,0,1,8.715574e-02,0.996195,-0.010771,0.999942
3,2017-12-31 00:30:00+00:00,-5.357727,23.148729,47.901936,45.736893,43.699357,0.0,77.119133,64.204399,1751.715563,...,0,0,0,0,0,1,1.305262e-01,0.991445,-0.010651,0.999943
4,2017-12-31 00:40:00+00:00,-5.357727,23.148729,47.901936,45.736893,43.699357,0.0,77.119133,64.204399,1751.715563,...,0,0,0,0,0,1,1.736482e-01,0.984808,-0.010532,0.999945


In [21]:
df.drop(["Time", "date", "WTG"], axis=1, inplace=True)

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118224 entries, 0 to 118223
Data columns (total 30 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   ActivePower                   118224 non-null  float64
 1   AmbientTemperatue             118224 non-null  float64
 2   BearingShaftTemperature       118224 non-null  float64
 3   Blade1PitchAngle              118224 non-null  float64
 4   Blade2PitchAngle              118224 non-null  float64
 5   ControlBoxTemperature         118224 non-null  float64
 6   GearboxBearingTemperature     118224 non-null  float64
 7   GearboxOilTemperature         118224 non-null  float64
 8   GeneratorRPM                  118224 non-null  float64
 9   GeneratorWinding1Temperature  118224 non-null  float64
 10  GeneratorWinding2Temperature  118224 non-null  float64
 11  HubTemperature                118224 non-null  float64
 12  MainBoxTemperature            118224 non-nul